# **PROJEK UAS PPW PART 1**
Amallia Tiara Putri (200411100025)


## **CRAWLING BERITA DARI WEB ONLINE**

### Import Modul
Modul yang digunakan adalah kombinasi dari tiga modul yaitu import requests
from bs4 import BeautifulSoup
import pandas as pd, yang dapat digunakan untuk mengambil data dari sebuah situs web (menggunakan requests), kemudian mengekstrak informasi yang diinginkan dari HTMLnya (menggunakan BeautifulSoup), dan akhirnya menyimpan hasilnya dalam bentuk dataframe Pandas (menggunakan pd).

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### FUNGSI get_data

In [ ]:
def get_data(url, category):
  data_page = requests.get(url)
  print(url)
  if data_page.status_code == 200:
    data_berita = []
    container = BeautifulSoup(data_page.content, 'html.parser')
    container_page = container.find('div', class_='detail__body-text itp_bodycontent')
    data = container_page.find_all('p')
    judul_berita = container.find('h1', class_='detail__title').text.strip()
    tanggal_berita = container.find('div', class_='detail__date').text.strip()

    data_content = ''.strip()
    for i in data:
      if  'ADVERTISEMENT' not in i.text.strip() and  'SCROLL TO RESUME CONTENT' not in i.text.strip() and i.find('div', class_='detail__multiple') == None:
        data_content += i.text.strip() + ' '

    data_berita.append(judul_berita)
    data_berita.append(tanggal_berita)
    data_berita.append(data_content)
    data_berita.append(category)

    return data_berita

Tujuan dari fungsi ini adalah untuk mengikis data dari halaman web yang ditentukan oleh urlparameter. Berikut rincian kodenya:

1. data_page = requests.get(url): Ini mengirimkan permintaan HTTP GET ke URL yang ditentukan menggunakan requestsperpustakaan dan menyimpan respons dalam data_pagevariabel.

2. print(url): Mencetak URL ke konsol.

3. if data_page.status_code == 200:: Memeriksa apakah kode status respons HTTP adalah 200 (OK), yang menunjukkan permintaan berhasil.

4. Di dalam blok bersyarat:

  - Ini menginisialisasi daftar kosong yang disebut data_berita.
  - Ia menggunakan BeautifulSoup untuk mengurai konten HTML halaman web (data_page.content).
  - Ia menemukan wadah konten berita menggunakan container.find('div', class_='detail__body-text itp_bodycontent').
  - Ini mengekstrak judul berita ( judul_berita) dan tanggal publikasi ( tanggal_berita).
  - Ini menginisialisasi string kosong yang disebut data_content.
  Itu mengulangi semua paragraf (p) dalam wadah dan menambahkan teksnya data_contentjika tidak mengandung kata kunci tertentu seperti 'IKLAN' dan 'GULIR UNTUK MERESUMKAN KONTEN'. Selain itu, ia melewatkan paragraf yang berisi kelas tertentu ('detail__multiple').
  - Itu menambahkan judul, tanggal, konten, dan kategori ke daftar data_berita.
5. Terakhir, ia mengembalikan data_beritadaftar yang berisi informasi yang diekstraksi.

### Fungsi get_content

In [ ]:
def get_content (url, category):
  menu_page = requests.get(url)
  if menu_page.status_code == 200:
    content = BeautifulSoup(menu_page.content, 'html.parser')
    content_page = content.find('div', class_='grid-row list-content')
    container_content = content_page.find_all('div', class_='media media--left media--image-radius block-link')

    # Untuk menyimpan url content
    data_content = []
    for c in container_content:
      link_content = c.find('a', class_='media__link')
      subtitle = c.find('h2', class_='media__subtitle')
      if subtitle == None and '20.detik' not in str(link_content) and 'intermeso' not in str(link_content) and 'foto' not in str(link_content):
        data_berita = get_data(link_content.get('href'), category)
        data_content.append(data_berita)

    next_page = content.find_all('a', class_='pagination__item')
    for i in next_page:
      if 'Next' == i.text.strip():
        return (data_content, i.get('href'))

Digunakan untuk mengekstrak konten dari URL tertentu yang terkait dengan artikel berita.

- Ini mengirimkan permintaan HTTP GET ke yang ditentukan urlmenggunakan requestsperpustakaan.
- Memeriksa apakah kode status respons adalah 200 (menunjukkan permintaan berhasil).
- Jika permintaan berhasil, permintaan tersebut akan digunakan BeautifulSoupuntuk mengurai konten HTML halaman.
- Menemukan elemen div tertentu dengan kelas 'konten daftar baris kisi' yang kemungkinan berisi konten utama.
- Mengekstrak daftar sub-elemen dengan kelas 'media media--media kiri--tautan blok radius gambar' di dalam konten utama.
- Ulangi sub-elemen ini dan ekstrak informasi seperti tautan ke konten dan subjudul.
- Memeriksa kondisi terkait subtitle dan konten tautan untuk menyaring item yang tidak diinginkan.
- Memanggil get_datafungsi (yang tidak disediakan) untuk mendapatkan data tambahan berdasarkan tautan dan kategori yang diekstraksi.
- Menambahkan data yang diperoleh ke daftar bernama data_content.
- Mencari link penomoran halaman dengan kelas 'pagination__item' untuk memeriksa apakah ada halaman "Berikutnya".
- Jika halaman "Berikutnya" ditemukan, fungsi akan mengembalikan tupel yang berisi daftar data_contentdan URL halaman berikutnya.

### Fungsi scraping_detik_news dengan kategori "FOOD"


In [ ]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 1, category= ['Food']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Food.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

Kode yang diberikan mendefinisikan fungsi scraping_detik_newsyang mengambil data berita dari situs newa.detik.com dan menyimpannya ke file CSV. Berikut rincian fungsionalitas fungsi tersebut:

1. Mengambil Beranda news.detik.com:
  - Fungsi ini mengambil tiga parameter: url, num_content, dan category. urlmenentukan URL beranda Detik.com, num_contentmenunjukkan jumlah artikel berita yang akan dikikis, dan categorymerupakan daftar kategori untuk memfilter artikel berita.
  - Fungsi mengirimkan permintaan GET ke yang ditentukan urlmenggunakan requestsperpustakaan. Jika permintaan berhasil ( response.status_code == 200), konten HTML diambil menggunakan response.content.

2. Mengurai Konten HTML:
  - Konten HTML diurai menggunakan BeautifulSoup menjadi objek sup. Objek sup mewakili struktur HTML halaman web yang diurai.
  - Fungsinya mencari menu navigasi menggunakan find()metode dengan kelas CSS 'static-nav'. Menu navigasi berisi link ke berbagai kategori berita.

3. Mengekstrak Tautan Navbar:

  - Fungsi ini mengulangi tautan yang ditemukan di menu navigasi dan mengekstrak teks tautan dan URL masing-masing menggunakan link.find('div').text.strip()dan link.get('href').
  - Informasi tautan yang diekstraksi disimpan dalam daftar bernama save_link_navbar.

4. Mengikis Artikel Berita:
  - Fungsi ini mengulangi tautan navbar yang diekstraksi dan memfilternya berdasarkan category.
  - Untuk setiap kategori yang dipilih, fungsi ini memanggil get_content()artikel berita untuk kategori tersebut. get_content()mengembalikan daftar artikel berita dan URL halaman berikutnya.
  - Artikel berita yang diambil akan ditambahkan ke data_semua_beritadaftar.

5. Menggores Halaman Berikutnya:
  - Fungsi ini mengekstrak URL halaman berikutnya dari variabel next_page.
  - Ini mengulangi num_content-1waktu, memanggil get_content()untuk mengambil artikel berita dari halaman berikutnya dan menambahkannya ke daftar data_semua_berita.

6. Menyimpan Data yang Tergores:
  - Fungsi ini mengubah data berita yang diambil menjadi Pandas DataFrame.
  - DataFrame disimpan ke file CSV bernama Data Berita Detik.csv.
  - Sebuah pesan dicetak yang menunjukkan keberhasilan penyimpanan data dengan jumlah artikel yang tergores.

Singkatnya, kode ini secara efektif mengambil artikel berita dari Detik.com berdasarkan kategori tertentu dan menyimpan data yang diambil ke file CSV.

### Fungsi scraping_detik_news dengan kategori "FINANCE"


In [ ]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 1, category= ['Finance']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Finance.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

### Fungsi scraping_detik_news dengan kategori "SEPAKBOLA"


In [ ]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 1, category= ['Sepakbola']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Sepakbola.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

### Crawling Data Kategori Food

In [ ]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=25, category=['Food'])

https://food.detik.com/info-kuliner/d-7087687/pengunjung-ini-usir-pengantar-makanan-yang-sedang-tunggu-pesanan-di-kafe
https://food.detik.com/berita-boga/d-7087667/rayakan-natal-dengan-menara-macaroon-hingga-bbq-seru-di-tengah-kota
https://food.detik.com/info-kuliner/d-7087998/12-rekomendasi-all-you-can-eat-paling-favorit-worth-it-di-bogor
https://food.detik.com/info-kuliner/d-7087969/iseng-pria-ini-masak-udang-mentah-di-toilet-pesawat
https://food.detik.com/info-kuliner/d-7087531/ngakak-pria-ini-pernah-rebutan-bihun-goreng-sama-bunga-citra-lestari
https://food.detik.com/info-kuliner/d-7087163/5-makanan-viral-sepanjang-tahun-2023-sudah-pernah-cicip
https://food.detik.com/berita-boga/d-7087119/kisah-jamu-gendong-dari-minuman-kerajaan-hingga-tercatat-di-relief-candi
https://food.detik.com/tempat-makan/d-7087101/begini-lezatnya-tahu-pong-semarang-favorit-orang-terkaya-di-indonesia
https://food.detik.com/info-kuliner/d-7086660/5-pesanan-starbucks-yang-tak-disukai-barista-apa-saja
https://f

### Crawling Data Kategori Finance

In [ ]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=25, category=['Finance'])

https://finance.detik.com/berita-ekonomi-bisnis/d-7088270/tiktok-shop-belum-sesuai-aturan-kemenkopukm-seharusnya-tidak-boleh
https://finance.detik.com/infrastruktur/d-7088225/ri-butuh-rp-500-t-untuk-rampungkan-proyek-jalan-tol-sampai-2024
https://finance.detik.com/berita-ekonomi-bisnis/d-7088183/bos-id-food-ramal-harga-gula-turun-hingga-awal-2024
https://finance.detik.com/energi/d-7088124/minyak-sudah-sentuh-us-70-barel-harga-pertalite-kapan-turun
https://finance.detik.com/berita-ekonomi-bisnis/d-7088100/berapa-gaji-tsamara-amany-yang-jadi-staf-khusus-erick-thohir
https://finance.detik.com/industri/d-7088081/upaya-pupuk-kaltim-dorong-transformasi-hijau-industri-pupuk-petrokimia
https://finance.detik.com/berita-ekonomi-bisnis/d-7088080/jelang-libur-nataru-pesanan-kamar-hotel-di-bali-yogyakarta-nyaris-penuh
https://finance.detik.com/bursa-dan-valas/d-7088047/bos-otorita-ikn-resmi-jadi-komut-ptpp
https://finance.detik.com/energi/d-7088003/jelang-akhir-tahun-dirjen-migas-waswas-kuota-solar

### Crawling Data Kategori Sepakbola

In [ ]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=25, category=['Sepakbola'])

https://sport.detik.com/sepakbola/bola-dunia/d-7088224/disambangi-psg-klub-divisi-6-prancis-ini-harus-cari-stadion-dulu
https://sport.detik.com/sepakbola/liga-indonesia/d-7088230/satgas-antimafia-ungkap-upaya-match-fixing-rumah-judi-online
https://sport.detik.com/sepakbola/liga-inggris/d-7087983/3-perbedaan-antara-abramovich-dan-boehly-saat-awal-pimpin-chelsea
https://sport.detik.com/sepakbola/uefa/d-7087979/eksperimen-baru-arteta-declan-rice-jadi-bek-tengah
https://sport.detik.com/sepakbola/liga-spanyol/d-7087835/viral-bellingham-ogah-tukar-jersey-dengan-pemain-lawan-faktanya
https://sport.detik.com/sepakbola/uefa/d-7087833/bukti-copenhagen-bukan-cuma-tim-penghibur
https://sport.detik.com/sepakbola/liga-inggris/d-7087753/paul-scholes-performa-man-united-menurun-tajam
https://sport.detik.com/sepakbola/uefa/d-7087659/sevilla-tersingkir-di-liga-champions-terbenam-di-laliga
https://sport.detik.com/sepakbola/uefa/d-7087540/newcastle-menanti-keajaiban-liga-champions
https://sport.detik.com/

### Penggabungan Antar Dokumen

In [10]:
food = pd.read_csv("/content/drive/MyDrive/PPW/tugas1/Data Berita Food.csv")
food

,Judul,Tanggal,Artikel,Category
0,"Baru Jualan Pastry, Wanita Ini Sudah Rugi Rp 3...","Senin, 11 Des 2023 19:30 WIB",Hal miris menimpa penjual makanan kaki lima in...,Food
1,Kocak! Ibu Ini Bawakan Kecap untuk Anaknya Pak...,"Senin, 11 Des 2023 19:00 WIB","Aksi seorang ibu menarik perhatian. Pasalnya, ...",Food
2,"Cicip Durian di Malaysia, Aktris China Fan Bin...","Senin, 11 Des 2023 18:30 WIB",Aktris China ternama Fan Bingbing baru-baru in...,Food
3,"Makan di LRT Berasa 'Piknik', Pria Ini Bikin H...","Senin, 11 Des 2023 18:00 WIB",Seorang pria di Malaysia menghebohkan karena a...,Food
4,Harga Seblak Lebih dari Rp 40 Ribu Disebut Mah...,"Senin, 11 Des 2023 17:30 WIB",Kisaran harga seblak di atas Rp 40 ribu untuk ...,Food
...,...,...,...,...
327,"3 Resep Sayuran Goreng Tepung yang Renyah, Lau...","Jumat, 24 Nov 2023 09:00 WIB","Tak hanya ditumis, sayuran juga enak digoreng ...",Food
328,Ini Dia 5 Sarapan Paling Sehat Menurut Ahli Gizi,"Jumat, 24 Nov 2023 08:00 WIB","Menu sarapan amat beragam, tapi kalau mau yang...",Food
329,"Resep Nasi Goreng Kari dan Terong, Aromatik da...","Jumat, 24 Nov 2023 07:00 WIB",Paduan bumbu kari dan terong membuat nasi gore...,Food
330,"Oatmeal Populer sebagai Makanan Sehat, Ini Fak...","Jumat, 24 Nov 2023 06:00 WIB",Oatmeal dikenal enak dan sehat. Oatmeal pun se...,Food


In [11]:
finance = pd.read_csv("/content/drive/MyDrive/PPW/tugas1/Data Berita Finance fix.csv")
finance

,Judul,Tanggal,Artikel,Category
0,"Transmart Diskon 20% Tiap Hari, Anggur Muscat ...","Senin, 11 Des 2023 22:00 WIB",Transmart diskon 20% setiap hari untuk memenuh...,Finance
1,"Pastikan Food Estate di Kalteg Jalan, Mentan-W...","Senin, 11 Des 2023 21:30 WIB",Menteri Pertanian (Mentan) Andi Amran Sulaiman...,Finance
2,"Ganjar Janji Geber Hilirisasi Industri, Singgu...","Senin, 11 Des 2023 21:18 WIB",Calon Presiden Ganjar Pranowo menyinggung soal...,Finance
3,"Diskon 20% Tiap Hari di Transmart, Semangka Me...","Senin, 11 Des 2023 21:00 WIB",Transmart menawarkan diskon 20% setiap hari un...,Finance
4,"IMF Adalah: Pengertian, Sejarah, Fungsi, dan T...","Senin, 11 Des 2023 20:45 WIB",IMF adalah singkatan dari International Moneta...,Finance
...,...,...,...,...
459,Intip Kisaran Gaji Pilot di Indonesia Beserta ...,"Selasa, 05 Des 2023 16:15 WIB",Pilot termasuk salah satu profesi idaman bagi ...,Finance
460,Sistem Bayar Tol Tanpa Setop Diklaim Bisa Dete...,"Selasa, 05 Des 2023 16:06 WIB",Penerapan sistem bayar tol tanpa setop alias M...,Finance
461,Lucu Nih... Ngerjain Penipu 'Kerja Freelance' ...,"Selasa, 05 Des 2023 16:01 WIB",Penipuan dengan modus menawarkan kerja freelan...,Finance
462,Jokowi Senang Harga Cabai di NTT Jauh Lebih Mu...,"Selasa, 05 Des 2023 15:55 WIB",Presiden Joko Widodo (Jokowi) melakukan peninj...,Finance


In [12]:
sepakbola = pd.read_csv('/content/drive/MyDrive/PPW/tugas1/Data Berita Sepakbola.csv')
sepakbola

,Judul,Tanggal,Artikel,Category
0,Jadwal Piala Dunia Antarklub 2023,"Senin, 11 Des 2023 22:00 WIB",Piala Dunia Antarklub 2023 bakal segera bergul...,Sepakbola
1,"Bisa Menang di Kandang Barcelona, Girona Tak B...","Senin, 11 Des 2023 21:40 WIB",Girona mampu memetik kemenangan meyakinkan saa...,Sepakbola
2,"Mengapa Anthony Martial Masih Jadi Starter, Te...","Senin, 11 Des 2023 21:20 WIB",Keputusan Erik ten Hag untuk memainkan Anthony...,Sepakbola
3,Bruno Fernandes Dianggap Kapten yang Buruk Unt...,"Senin, 11 Des 2023 20:45 WIB",Chris Sutton menganggap Bruno Fernandes adalah...,Sepakbola
4,Nasib Pioli Akan Ditentukan Hasil Newcastle Vs...,"Senin, 11 Des 2023 20:30 WIB",Nasib pelatih AC Milan Stefano Pioli di ujung ...,Sepakbola
...,...,...,...,...
547,Barcelona Vs Atletico: Ajang Pembuktian Joao F...,"Jumat, 01 Des 2023 16:30 WIB",Barcelona vs Atletico Madrid bakal jadi laga y...,Sepakbola
548,MU Sedang Sulit Kalahkan Newcastle: Cuma Sekal...,"Jumat, 01 Des 2023 16:00 WIB",Konsistensi Manchester United di Liga Inggris ...,Sepakbola
549,10 Fakta Napoli Vs Inter Milan,"Jumat, 01 Des 2023 15:40 WIB",Napoli akan menjamu Inter Milan di Liga Italia...,Sepakbola
550,Bellingham Dibandingkan sama Zidane? Ini Kata ...,"Jumat, 01 Des 2023 15:40 WIB",Banyak pihak yang menyamakan Jude Bellingham d...,Sepakbola


In [13]:
hasil1 = pd.DataFrame(food, columns=['Judul', 'Artikel', 'Category'])
hasil2 = pd.DataFrame(finance, columns=['Judul', 'Artikel', 'Category'])
hasil3 = pd.DataFrame(sepakbola, columns=['Judul', 'Artikel', 'Category'])

In [14]:
combined_df = pd.concat([hasil1, hasil2, hasil3]).reset_index(drop=True)
combined_df

,Judul,Artikel,Category
0,"Baru Jualan Pastry, Wanita Ini Sudah Rugi Rp 3...",Hal miris menimpa penjual makanan kaki lima in...,Food
1,Kocak! Ibu Ini Bawakan Kecap untuk Anaknya Pak...,"Aksi seorang ibu menarik perhatian. Pasalnya, ...",Food
2,"Cicip Durian di Malaysia, Aktris China Fan Bin...",Aktris China ternama Fan Bingbing baru-baru in...,Food
3,"Makan di LRT Berasa 'Piknik', Pria Ini Bikin H...",Seorang pria di Malaysia menghebohkan karena a...,Food
4,Harga Seblak Lebih dari Rp 40 Ribu Disebut Mah...,Kisaran harga seblak di atas Rp 40 ribu untuk ...,Food
...,...,...,...
1343,Barcelona Vs Atletico: Ajang Pembuktian Joao F...,Barcelona vs Atletico Madrid bakal jadi laga y...,Sepakbola
1344,MU Sedang Sulit Kalahkan Newcastle: Cuma Sekal...,Konsistensi Manchester United di Liga Inggris ...,Sepakbola
1345,10 Fakta Napoli Vs Inter Milan,Napoli akan menjamu Inter Milan di Liga Italia...,Sepakbola
1346,Bellingham Dibandingkan sama Zidane? Ini Kata ...,Banyak pihak yang menyamakan Jude Bellingham d...,Sepakbola


### Menyimpan Dataset

In [17]:
combined_df.to_csv('Dataset_Berita_UAS.csv')